In [15]:
def generate_random_map(size, p):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] !='H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]

generate_random_map(5,0.7)

['SHFFH', 'FFFHF', 'FFFHF', 'HHFHH', 'FFFFG']

In [16]:
custom_map = ['SFFFF', 'FHFFF', 'FFFFH', 'HFHFH', 'FHFFG']
gamma= 0.99
eps=0.0001

In [17]:
import numpy as np
import gym

def eval_state_action(V, s, a, gamma=gamma):
    return np.sum([p * (rew + gamma*V[next_s]) for p, next_s, rew, _ in env.P[s][a]])

def policy_evaluation(V, policy, eps=eps):
    '''
    Policy evaluation. Update the value function until it reach a steady state
    '''
    while True:
        delta = 0
        # loop over all states
        for s in range(nS):
            old_v = V[s]
            # update V[s] using the Bellman equation
            V[s] = eval_state_action(V, s, policy[s])
            delta = max(delta, np.abs(old_v - V[s]))

        if delta < eps:
            break

def policy_improvement(V, policy):
    '''
    Policy improvement. Update the policy based on the value function
    '''
    policy_stable = True
    for s in range(nS):
        old_a = policy[s]
        # update the policy with the action that bring to the highest state value
        policy[s] = np.argmax([eval_state_action(V, s, a) for a in range(nA)])
        if old_a != policy[s]: 
            policy_stable = False

    return policy_stable


def run_episodes(env, policy, num_games=1000):
    '''
    Run some games to test a policy
    '''
    tot_rew = 0
    state = env.reset()

    for _ in range(num_games):
        done = False
        while not done:
            # select the action accordingly to the policy
            next_state, reward, done, _ = env.step(policy[state])
                
            state = next_state
            tot_rew += reward 
            if done:
                state = env.reset()

    print('Won %i of %i games!'%(tot_rew, num_games))

            
if __name__ == '__main__':
    # create the environment
    #desc= custom_map  or desc= generate_random_map(size,p) 
    env = gym.make('FrozenLake-v0',desc= custom_map)
    # enwrap it to have additional information from it
    env = env.unwrapped
    env.render()

    # spaces dimension
    nA = env.action_space.n
    nS = env.observation_space.n
    
    # initializing value function and policy
    V = np.zeros(nS)
    policy = np.zeros(nS)

    # some useful variable
    policy_stable = False
    it = 0

    while not policy_stable:
        policy_evaluation(V, policy)
        policy_stable = policy_improvement(V, policy)
        it += 1

    print('Converged after %i policy iterations'%(it))
    run_episodes(env, policy)
    print(V.reshape((s,s)))
    print(policy.reshape((s,s)))


SFFFF
FHFFF
FFFFH
HFHFH
FHFFG
Converged after 6 policy iterations
Won 262 of 1000 games!
[[0.15541242 0.16039381 0.17038269 0.17685614 0.17418432]
 [0.13330009 0.         0.16925812 0.18507288 0.1769174 ]
 [0.11537357 0.10104916 0.15754923 0.20715243 0.        ]
 [0.         0.03334622 0.         0.28512501 0.        ]
 [0.         0.         0.32352941 0.65686275 0.        ]]
[[3. 3. 2. 2. 0.]
 [0. 0. 2. 2. 3.]
 [3. 1. 3. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0.]]


In [25]:
import numpy as np
import gym

def eval_state_action(V, s, a, gamma=gamma):
    return np.sum([p * (rew + gamma*V[next_s]) for p, next_s, rew, _ in env.P[s][a]])

def value_iteration(eps=eps):
    '''
    Value iteration algorithm
    '''
    V = np.zeros(nS)
    it = 0

    while True:
        delta = 0
        # update the value of each state using as "policy" the max operator
        for s in range(nS):
            old_v = V[s]
            V[s] = np.max([eval_state_action(V, s, a) for a in range(nA)])
            delta = max(delta, np.abs(old_v - V[s]))

        if delta < eps:
            break
        else:
            print('Iter:', it, ' delta:', np.round(delta, 5))
        it += 1

    return V

def run_episodes(env, V, num_games=1000):
    '''
    Run some test games
    '''
    tot_rew = 0
    state = env.reset()

    for _ in range(num_games):
        done = False
        while not done:
            action = np.argmax([eval_state_action(V, state, a) for a in range(nA)])
            next_state, reward, done, _ = env.step(action)

            state = next_state
            tot_rew += reward 
            if done:
                state = env.reset()

    print('Won %i of %i games!'%(tot_rew, num_games))

            
if __name__ == '__main__':
    # create the environment
    #desc= custom_map  or desc= generate_random_map(size,p) 
    s=5
    env = gym.make('FrozenLake-v0',desc= custom_map)
    # enwrap it to have additional information from it
    env = env.unwrapped
    env.render()

    # spaces dimension
    nA = env.action_space.n
    nS = env.observation_space.n

    # Value iteration
    V = value_iteration(eps=0.0001)
    # test the value function on 100 games
    run_episodes(env, V, 1000)
    # print the state values
    print(V.reshape((5,5)))
    print(policy.reshape((5,5)))


SFFFF
FHFFF
FFFFH
HFHFH
FHFFG
Iter: 0  delta: 0.33333
Iter: 1  delta: 0.1463
Iter: 2  delta: 0.08458
Iter: 3  delta: 0.05305
Iter: 4  delta: 0.03158
Iter: 5  delta: 0.01851
Iter: 6  delta: 0.01114
Iter: 7  delta: 0.0099
Iter: 8  delta: 0.00928
Iter: 9  delta: 0.00901
Iter: 10  delta: 0.00869
Iter: 11  delta: 0.0082
Iter: 12  delta: 0.00765
Iter: 13  delta: 0.00708
Iter: 14  delta: 0.00652
Iter: 15  delta: 0.006
Iter: 16  delta: 0.0058
Iter: 17  delta: 0.00561
Iter: 18  delta: 0.00555
Iter: 19  delta: 0.00543
Iter: 20  delta: 0.00526
Iter: 21  delta: 0.00507
Iter: 22  delta: 0.00485
Iter: 23  delta: 0.00462
Iter: 24  delta: 0.00439
Iter: 25  delta: 0.00416
Iter: 26  delta: 0.00393
Iter: 27  delta: 0.00371
Iter: 28  delta: 0.0035
Iter: 29  delta: 0.00331
Iter: 30  delta: 0.00313
Iter: 31  delta: 0.00296
Iter: 32  delta: 0.00281
Iter: 33  delta: 0.00266
Iter: 34  delta: 0.00252
Iter: 35  delta: 0.00238
Iter: 36  delta: 0.00226
Iter: 37  delta: 0.00214
Iter: 38  delta: 0.00203
Iter: 39  d